In [2]:
def complex_mult(a_r, a_i, b_r, b_i):
    """Complex multiplication: (a_r + a_i*j) * (b_r + b_i*j)"""
    real_part = a_r * b_r - a_i * b_i
    imag_part = a_r * b_i + a_i * b_r
    return real_part, imag_part

def butterfly(in1_r, in1_i, in2_r, in2_i, w_r, w_i):
    """Butterfly operation for FFT"""
    out1_r = in1_r + in2_r
    out1_i = in1_i + in2_i

    z1_r = in1_r - in2_r
    z1_i = in1_i - in2_i

    out2_r, out2_i = complex_mult(z1_r, z1_i, w_r, w_i)

    return out1_r, out1_i, out2_r, out2_i

# Test Case
def test_butterfly():
    in1_r = 10
    in1_i = 5
    in2_r = 3
    in2_i = 2
    w_r = 2
    w_i = 3

    out1_r, out1_i, out2_r, out2_i = butterfly(in1_r, in1_i, in2_r, in2_i, w_r, w_i)

    print(f"out1_r: {out1_r}, out1_i: {out1_i}, out2_r: {out2_r}, out2_i: {out2_i}")

    in2_r = 8
    in2_i = 4
     
    out1_r, out1_i, out2_r, out2_i = butterfly(in1_r, in1_i, in2_r, in2_i, w_r, w_i)

    print(f"out1_r: {out1_r}, out1_i: {out1_i}, out2_r: {out2_r}, out2_i: {out2_i}")
    
# Run the test case
test_butterfly()

out1_r: 13, out1_i: 7, out2_r: 5, out2_i: 27
out1_r: 18, out1_i: 9, out2_r: 1, out2_i: 8


Butterfly works
<br />
iverilog output:

`
out1_r:    13, out1_i:     7, out2_r:     5, out2_i:    27
out1_r:    18, out1_i:     9, out2_r:     1, out2_i:     8
testbench.sv:58: $finish called at 20 (1s)
`

In [1]:
import numpy as np

def complex_mult(a_r, a_i, b_r, b_i):
    """Complex multiplication: (a_r + a_i*j) * (b_r + b_i*j)"""
    real_part = a_r * b_r - a_i * b_i
    imag_part = a_r * b_i + a_i * b_r
    return real_part, imag_part

def butterfly(in1_r, in1_i, in2_r, in2_i, w_r, w_i):
    """Butterfly operation for FFT"""
    out1_r = in1_r + in2_r
    out1_i = in1_i + in2_i

    z1_r = in1_r - in2_r
    z1_i = in1_i - in2_i

    out2_r, out2_i = complex_mult(z1_r, z1_i, w_r, w_i)

    return out1_r, out1_i, out2_r, out2_i

def four_point_fft(x_r, x_i):
    """4-point FFT"""
    # First stage
    stage1_0_r, stage1_0_i, stage1_1_r, stage1_1_i = butterfly(x_r[0], x_i[0], x_r[2], x_i[2], 1, 0)
    stage1_2_r, stage1_2_i, stage1_3_r, stage1_3_i = butterfly(x_r[1], x_i[1], x_r[3], x_i[3], 1, 0)

    # Second stage
    y0_r, y0_i, y1_r, y1_i = butterfly(stage1_0_r, stage1_0_i, stage1_2_r, stage1_2_i, 1, 0)
    y2_r, y2_i, y3_r, y3_i = butterfly(stage1_1_r, stage1_1_i, stage1_3_r, stage1_3_i, 0, -1)

    return [y0_r, y1_r, y2_r, y3_r], [y0_i, y1_i, y2_i, y3_i]

# Test Cases
def test_four_point_fft():
    # Initialize inputs
    x_r = [10, 8, 6, 4]
    x_i = [0, 0, 0, 0]

    # Expected output using numpy
    x_complex = np.array(x_r) + 1j * np.array(x_i)
    fft_result = np.fft.fft(x_complex)

    # Get the FFT output from the custom function
    y_r, y_i = four_point_fft(x_r, x_i)

    # Display results
    for i in range(4):
        print(f"Expected y{i}_r: {fft_result[i].real:.2f}, y{i}_i: {fft_result[i].imag:.2f}")
        print(f"Computed y{i}_r: {y_r[i]:.2f}, y{i}_i: {y_i[i]:.2f}")

    # Add more test cases if needed
    x_r = [10, 8, 6, 4]
    x_i = [5, 4, 3, 2]

    x_complex = np.array(x_r) + 1j * np.array(x_i)
    fft_result = np.fft.fft(x_complex)

    y_r, y_i = four_point_fft(x_r, x_i)

    for i in range(4):
        print(f"Expected y{i}_r: {fft_result[i].real:.2f}, y{i}_i: {fft_result[i].imag:.2f}")
        print(f"Computed y{i}_r: {y_r[i]:.2f}, y{i}_i: {y_i[i]:.2f}")

# Run the test cases
test_four_point_fft()

Expected y0_r: 28.00, y0_i: 0.00
Computed y0_r: 28.00, y0_i: 0.00
Expected y1_r: 4.00, y1_i: -4.00
Computed y1_r: 4.00, y1_i: 0.00
Expected y2_r: 4.00, y2_i: 0.00
Computed y2_r: 8.00, y2_i: 0.00
Expected y3_r: 4.00, y3_i: 4.00
Computed y3_r: 0.00, y3_i: 0.00
Expected y0_r: 28.00, y0_i: 14.00
Computed y0_r: 28.00, y0_i: 14.00
Expected y1_r: 6.00, y1_i: -2.00
Computed y1_r: 4.00, y1_i: 2.00
Expected y2_r: 4.00, y2_i: 2.00
Computed y2_r: 8.00, y2_i: 4.00
Expected y3_r: 2.00, y3_i: 6.00
Computed y3_r: 0.00, y3_i: 0.00
